<a href="https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這支 notebook 是為了選取剪輯 news-digest 的部分新聞分類，以利後續轉貼
執行步驟如下：
1. 按下 Install 下那個 code cell 前面的 ► 按鈕來安裝 clip 模組.
2. 根據需要，填表執行後面的 Forms 或 Widgets UI

### Install 

In [ ]:
!wget https://raw.githubusercontent.com/YorkJong/news-digest/main/api/clip.py
!wget https://raw.githubusercontent.com/YorkJong/news-digest/main/api/line.py

from clip import *

--2023-03-23 03:32:17--  https://raw.githubusercontent.com/YorkJong/news-digest/main/api/clip.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5458 (5.3K) [text/plain]
Saving to: ‘clip.py’

clip.py             100%[===================>]   5.33K  --.-KB/s    in 0s      

2023-03-23 03:32:18 (50.2 MB/s) - ‘clip.py’ saved [5458/5458]

--2023-03-23 03:32:18--  https://raw.githubusercontent.com/YorkJong/news-digest/main/api/line.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3425 (3.3K) [text/plain]
Saving to: ‘line.py’

line.py 

### Select Categories with the Colab Forms

In [ ]:
#@title Single Selection { run: "auto" }
category = "Crypto" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
with_hashtags = False #@param {type:"boolean"}
with_headers = False #@param {type:"boolean"}

fn = get_latest_fn("journals")
content = get_file(fn)
lines = get_lines_of_categories([category], content, with_hashtags, with_headers)
text = '\n'.join(lines)
print(text)

- [Raoul Pal：12个月内，标普、纳指都将创出新高，加密货币会更快！比特币这轮更像2013年，而非2019年！](https://www.youtube.com/watch?v=K0UU0WnyPW4)
- [OpenAI創辦人項目》世界幣 Worldcoin 將向10億人空投？一覽世界級的Web3計畫](https://www.blocktempo.com/worldcoin-will-airdrop-to-the-1b-users-of-whole-world/)
- [BTC漲破2.8萬，比特幣「礦工爽快回血」！收入暴增36%](https://www.blocktempo.com/bitcoin-rally-is-pure-gravy-for-miners-finally-seeing-a-light-at-the-end-of-the-tunnel/)
- [比特幣100萬鎂賭局》Coinbase前CTO：我不是要賺錢！Fed瘋狂印鈔讓美元一文不值](https://www.blocktempo.com/former-coinbase-ctos-million-dollar-gamble/)
- [經典虛擬寵物遊戲》Neopets Meta：轉生 Web3，宣布轉至 Polygon 鏈](https://www.blocktempo.com/neopets-metaverse-transition-from-web2-to-web3/)
- [ChatGPT 檢查程式碼大失誤！Coinbase：多次忽略高風險BUG，無法取代人工](https://www.blocktempo.com/coinbae-publish-token-security-with-chatgpt/)
-


In [ ]:
#@title Multiple Selections { run: "auto" }
first_category = "Tesla & SpaceX; Vehicle" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
last_category = "Finance" #@param ["Tesla & SpaceX; Vehicle", "Tech Titans", "Finance", "Taiwan", "Crypto", "Technology", "AI"] {allow-input: true}
with_hashtags = False #@param {type:"boolean"}
with_headers = True #@param {type:"boolean"}

fn = get_latest_fn("journals")
content = get_file(fn)
categories = get_categories(content)
categories = get_sublist(categories, first_category, last_category)
lines = get_lines_of_categories(categories, content, with_hashtags, with_headers)
text = '\n'.join(lines)
print(text)


### Tesla & SpaceX; Vehicle
- [「不需借光」，Tesla Powerwall家用儲能重新開放單買](https://www.digitimes.com.tw/tech/dt/n/shwnws.asp?cnlid=1&id=0000659463_ASL58FZN5IDRDA4LUB1LC)
- [特斯拉獲穆迪充電　股價勁升8%直衝200美元關　評級升至投資級別](https://inews.hket.com/article/3487919/【TSLA】特斯拉獲穆迪充電%E3%80%80股價勁升8-直衝200美元關%E3%80%80評級升至投資級別)
- [特斯拉(TSLA.US)Q1中國銷量或強勁：有望超10.69萬輛 作者 智通財經](https://m.hk.investing.com/news/stock-market-news/article-310687)
- [工程師、設計師、CEO 和產品經理，誰會最先被AI取代？馬斯克這樣選](https://www.bnext.com.tw/article/74508/placed-by-ai-sit-which-first-with-musk-ans)
- [降價助攻 特斯拉歐洲市占率攀升](https://www.chinatimes.com/amp/realtimenews/20230321005968-260410)
- [Tesla快充模組含「地板」？建置效率極高！連美國總統拜登也稱讚](https://today.line.me/tw/v2/amp/article/VxWeY9B)
- [AI晶片持續火爆 比亞迪也加入戰局](https://m.cnyes.com/news/id/5121294)
- [補貼退場買氣驟減，中國電車龍頭比亞迪砍班減產](https://technews.tw/2023/03/22/byd-reduces-work-and-production/)
- [Toyota氫能電動車Mirai將來台試運行　4月中旬智慧移動展首度亮相](https://tw.nextapple.com/gadget/20230322/62A4BA558BBDD7EFA8CCF1C2BFE97A99)
-
### Tech Titans
- [閃美禁令 輝達推中國特供版H

### Select multiple categories with Jupyter Widgets

#### Layout Widges

In [ ]:
import ipywidgets as widgets

fn = get_latest_fn("journals")
content = get_file(fn)
options = get_categories(content)

categories = widgets.SelectMultiple(
    options=options,
    value=[options[0]],
    rows=len(options),
    description='Categories',
    disabled=False
)

with_hashtags = widgets.Checkbox(
    value=True,
    description='with hashtags',
    disabled=False
)

with_headers = widgets.Checkbox(
    value=True,
    description='with headers',
    disabled=False
)

flags = widgets.VBox([with_hashtags, with_headers])
ui = widgets.HBox([categories, flags])

#### Show Widgets UI

In [ ]:
ui

In [ ]:
lines = get_lines_of_categories(categories.value, content, with_hashtags.value, with_headers.value)
text = '\n'.join(lines) 
print(text)

### Taiwan
- [台積電危險了？晶片法案再爆2恐怖條件](https://ctee.com.tw/news/global/829864.html) #TSM #semicon #TW
- [輝達攜 3 大廠加快微影製程、劍指 2 奈米，台積 ADR 嗨](https://technews.tw/2023/03/22/nvidia-eda-tsmc/) #NVDA #TSM #semicon
- [不敢相信！金融業沒踩AT1地雷 5國銀客戶卻握16億元恐成壁紙](https://m.cnyes.com/news/id/5121323) #TW
- [美晶片法案將限制在陸擴產　台積電、三星受衝擊](https://today.line.me/tw/v2/article/l2azlLB) #TSM #semicon
- [猴子不知屁股紅！自己都沒做好就急著喊電動車產業是下座護國神山？](https://technews.tw/2023/03/21/charging-station-policy/) #TW #EV
- [血本無歸 國內投資人踩雷瑞信AT1近17億](https://ctee.com.tw/news/finance/829428.html) #TW #CSGN
- [台積電要去德國設廠 外媒爆1大難題](https://ctee.com.tw/news/tech/829114.html) #TSM #Europ #TW
-


### Line Notify

In [ ]:
#@title Send message
receiver = "\u963F\u5BEC" #@param ["阿寬", "區塊鏈", "股之呼吸", "MiniGroup"]

tok_tbl = {
    '阿寬': 'LINE ACCESS TOKEN 1',
    '區塊鏈': 'LINE ACCESS TOKEN 2',
    '股之呼吸': 'LINE ACCESS TOKEN 3',
    'MiniGroup': 'LINE ACCESS TOKEN 4',
}

import line
message = f'\n\n{text}'
token = tok_tbl[receiver]
line.notify_message(message, token)

#### Scriptions

In [ ]:
import time

scriptions = { 
    (('Tesla & SpaceX; Vehicle', 'Tech Titans'), ('股之呼吸', 'MiniGroup')),
    (('Finance'), ('股之呼吸', 'MiniGroup')),
    (('Crypto',), ('股之呼吸', '區塊鏈')),
    (('AI'), ('股之呼吸', 'MiniGroup'))
}

fn = get_latest_fn("journals")
content = get_file(fn)

for topics, clients in scriptions:
    lines = get_lines_of_categories(topics, content)
    message = f'\n\n{'\n'.join(lines)}'
    for receiver in clients:
        token = tok_tbl[receiver]
        line.notify_message(message, token)
    time.sleep(3)

### References
* [LINE Notify ::My page](https://notify-bot.line.me/my/)